#### 해결하고자 하는 문제

1.   기상청 공공데이터를 통해 최신 지진 데이터를 수집하고
2.   수집한 지진 데이터를 이용해 어디 지역에서 지진이 자주 일어났는지 확인한 뒤
3.   우리가 주거하는 강원지역의 지진 데이터를 확인하고
4.   강원지역의 지진해일대피소의 위치를 확인해
5.   불시의 상황이 발생하더라도 이곳으로 대피할 수 하도록 한다.


#### 문제 선정이유

*   아직은 지진이 일본과 같이 수시로 찾아오진 않지만
*   한반도는 이미 지진 안전지대가 아니다.
*   따라서 어느 지역에서 지진이 많이 일어났는지 프로그래밍을 이용해 확인하고
*   지진이 일어나도 어느곳으로 대피해야 하는지 모르기 때문에
*   이번 프로그래밍을 통해 우리 지역의 가까운 대피소가 어디있는지 알아 지진으로 인한 피해를 줄일 수 있다.

#### 문제 분석 (문제 해결을 위해 필요한 요소 및 작업들)

*   3년간의 지진 데이터와 강원도의 대피소 정보를 CSV파일로 나타낸다.
*   3년간의 지진이 어디서 많이 일어났는지 빈도를 알기 위해 오픈api 지도를 이용해 지진이 발생한 위치를 나타낸다.
*   강원도에 있는 대피소 정보를 이용해 지도에 위치를 표시해 직관적으로 볼 수 있게 한다.

#### 해결 과정과 결과 (시행착오 및 소스코드)

*   오픈 API로 데이터를 불러올 때 현재로부터 이틀 전까지의 데이터만 나오고 데이터를 다 불러오지 못해서 이용하기가 어려웠다.
*   데이터에 빈 공간이 있어 마커 클러스터를 바로 적용하지 못했다.

#  : 소스 코드 짜기, csv파일 검색
#  : 파일 오류 확인 및 검토

## **기초 코드**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()
fm._rebuild()
plt.rc('font', family=font_name)

mpl.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'retina'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import folium

### **3년간 지진 데이터**

In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/DPL/3년간지진정보.csv",encoding='cp949')
df.head()

,번호,시간,규모,진앙(km),위도,경도,지역,위치
0,1,2017-10-04 2:11,2.4,22,35.56,129.88,울산,울산 동구 동쪽 42km 해역
1,2,2017-10-04 12:10,2.1,11,36.16,127.13,충남,충남 논산시 남동쪽 4km 지역
2,3,2017-10-08 12:18,2.8,16,34.84,127.25,전남,전남 보성군 동북동쪽 17km 지역
3,4,2017-10-13 1:41,2.7,3,41.39,129.03,북한,북한 함경북도 길주 북북서쪽 54km 지역
4,5,2017-10-16 6:08,2.8,18,33.86,125.30,전남,전남 신안군 흑산면 남쪽 89km 해역


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   번호      382 non-null    int64  
 1   시간      382 non-null    object 
 2   규모      382 non-null    float64
 3   진앙(km)  382 non-null    int64  
 4   위도      382 non-null    float64
 5   경도      382 non-null    float64
 6   지역      382 non-null    object 
 7   위치      382 non-null    object 
dtypes: float64(3), int64(2), object(3)
memory usage: 24.0+ KB


In [ ]:
df.shape

(382, 8)

In [ ]:
flag = df['지역'] =='충남'
flag2 = df['규모'] ==2.1
new_df = df[flag & flag2 ]
new_df

,번호,시간,규모,진앙(km),위도,경도,지역,위치
1,2,2017-10-04 12:10,2.1,11,36.16,127.13,충남,충남 논산시 남동쪽 4km 지역
189,190,2018-08-09 0:21,2.1,0,37.03,125.30,충남,충남 태안군 서격렬비도 북북서쪽 51km 해역
190,191,2018-08-09 0:23,2.1,0,37.03,125.27,충남,충남 태안군 서격렬비도 북북서쪽 53km 해역


## **지도로 보는 지진이 난 위치**

In [ ]:
m = folium.Map([36.019459265990136, 129.34334121295782])
folium.Marker([36.019459265990136, 129.34334121295782], popup='<i>포항 시청</i>', tooltip="포항은 안전해요").add_to(m)
m.save('/content/gdrive/My Drive/DPL/포항시청.html')
m

In [ ]:
df_map = pd.read_csv("/content/gdrive/My Drive/DPL/3년간지진정보.csv",encoding = 'cp949')
area = ['경북']

m = folium.Map([36.01936381158155, 129.34336267063014], zoom_start=30)

for i in df_map.index:
    if df_map.loc[i,'지역'] in area:
      sub_lat = df_map.loc[i, "위도"]
      sub_long = df_map.loc[i, "경도"]
      title = df_map.loc[i, "규모"]
      typ = df_map.loc[i,"규모"]


      icon_color = "red"


      folium.Marker([sub_lat, sub_long], icon=folium.Icon(color=icon_color), popup=f'<i>{typ}</i>', tooltip=title).add_to(m)

m.save('/content/gdrive/My Drive/DPL/춘천시 대피소.html')
m

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map([37.87493146323539, 127.71900194031704], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)

for i in df_map.index:
      sub_lat = df_map.loc[i, "위도"]
      sub_long = df_map.loc[i, "경도"]
      title = df_map.loc[i, "규모"]
      typ = df_map.loc[i,"규모"]

      icon_color = "red"

      folium.Marker([sub_lat, sub_long], icon=folium.Icon(color=icon_color), popup=f'<i>{typ}</i>', tooltip=title).add_to(marker_cluster)

m.save('/content/gdrive/My Drive/DPL/전국 지진.html')
m

## **강원도의 지진해일 대피소**

In [ ]:
import csv
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm


f = open('/content/gdrive/My Drive/DPL/강원도지진해일대피소.csv','r',encoding = 'cp949')
data = csv.reader(f)
header = next(data)
print(header)

area = input("출력을 원하는 지역(강원도 시군구)을 입력하세요 : ")
shel = []

for row in data:
       if row[13]=='Y' and area in row[2]:
         shel.append(row[3])
         shel.append(row[7])

for i,j in enumerate(shel):
  print(i+1,j)

['번호', '시도명', '시군구명', '지진해일대피소명', '지진해일대피소구분', '지진해일대피소유형', '지진해일대피소유형구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '수용가능면적', '최대수용인원수', '지진해일대피소운영상태', '지진해일대피소전화번호', '부대편의시설', '주민대피지구명', '주민대피지구내가구수', '주민대피지구내거주인수', '주민대피지구내재해약자수', '내진적용여부', '내진설계등급', '해안선이격거리', '해발높이', '지진대피안내표지판수', '긴급대피장소안내표지판수', '지진대피로안내표지판수', '관리기관명', '관리기관전화번호', '데이터기준일자', '제공기관코드', '제공기관명']
출력을 원하는 지역(강원도 시군구)을 입력하세요 : 춘천
1 상천초등학교 운동장
2 강원도 춘천시 신북읍 상천2길 19
3 오동초등학교 운동장
4 강원도 춘천시 신북읍 장본길 74
5 근화초등학교 운동장
6 강원도 춘천시 소양로163번길 23 (근화동)
7 후평초등학교 운동장
8 강원도 춘천시 춘천로295번길 17 (후평동)
9 부안초등학교 운동장
10 강원도 춘천시 서부대성로239번길 70 (후평동)
11 동부초등학교 운동장
12 강원도 춘천시 서부대성로 339 (후평동)
13 호반초등학교 운동장
14 강원도 춘천시 후만로126번길 35 (후평동)
15 천전초등학교 운동장
16 강원도 춘천시 신북읍 율문1길 16-9
17 춘성중학교 운동장
18 강원도 춘천시 신북읍 율문길 95-8
19 가산초등학교 운동장
20 강원도 춘천시 동면 가산로 52
21 만천초등학교 운동장
22 강원도 춘천시 동면 후석로326번길 89
23 동산중학교 운동장
24 강원도 춘천시 동산면 영서로 461-22
25 조양초등학교 운동장
26 강원도 춘천시 동산면 조양길 257
27 금병초등학교 운동장
28 강원도 춘천시 신동면 김유정로 1394
29 금산초등학교 운동장
30 강원도 춘천시 서면 금산2길 21
31 당림초등학교 운동장
32 강원도 춘천시 

In [ ]:
import requests, bs4
import json
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import unquote, quote_plus, urlencode
from xml.etree import ElementTree as ET
from datetime import datetime, timedelta, date

address = input("지진해일대피소를 도로명으로 입력해주세요 : ")


url = 'http://api.vworld.kr/req/address?service=address&request=GetCoord'
API_KEY='60821C27-D454-3290-A84E-07A4BED720B6'
My_API_Key = unquote(API_KEY)

queryParams = '&' + urlencode(
        {
            quote_plus('key') : My_API_Key,
            quote_plus('address') : address,
            quote_plus('type') : 'road',
            quote_plus('format') : 'json',
            quote_plus('crs') : 'epsg:4326'
        }
    )

print(url+queryParams)

response = requests.get(url + queryParams).text
print(response)
geo_info = json.loads(response)
latitude = geo_info['response']['result']['point']['y']
longitue = geo_info['response']['result']['point']['x']

print(latitude)
print(longitue)


지진해일대피소를 도로명으로 입력해주세요 : 강원도 춘천시 중앙로3가 공지로 442-10
http://api.vworld.kr/req/address?service=address&request=GetCoord&key=60821C27-D454-3290-A84E-07A4BED720B6&address=%EA%B0%95%EC%9B%90%EB%8F%84+%EC%B6%98%EC%B2%9C%EC%8B%9C+%EC%A4%91%EC%95%99%EB%A1%9C3%EA%B0%80+%EA%B3%B5%EC%A7%80%EB%A1%9C+442-10&type=road&format=json&crs=epsg%3A4326


ConnectionError: ignored

### **지도에서의 대피소 위치**

In [ ]:
m = folium.Map([37.87493146323539, 127.71900194031704])
folium.Marker([37.87493146323539, 127.71900194031704], popup='<i>대피소</i>', tooltip="대피소는 안전해요").add_to(m)
m.save('/content/gdrive/My Drive/DPL/대피소1.html')
m

In [ ]:
df_map = pd.read_csv("/content/gdrive/My Drive/DPL/강원도지진해일대피소.csv",encoding = 'cp949')
area = ['춘천시']

m = folium.Map([37.87493146323539, 127.71900194031704], zoom_start=30)

for i in df_map.index:
    if df_map.loc[i,'시군구명'] in area:
      sub_lat = df_map.loc[i, "위도"]
      sub_long = df_map.loc[i, "경도"]
      title = df_map.loc[i, "번호"]
      typ = df_map.loc[i,"내진적용여부"]

      icon_color = "blue"
      if df_map.loc[i, "내진적용여부"] == "미적용":
          icon_color = "red"
      if df_map.loc[i, "내진적용여부"] == "부분적용":
          icon_color = "green"


      folium.Marker([sub_lat, sub_long], icon=folium.Icon(color=icon_color), popup=f'<i>{typ}</i>', tooltip=title).add_to(m)

m.save('/content/gdrive/My Drive/DPL/춘천시 대피소.html')
m


In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map([37.87493146323539, 127.71900194031704], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)

for i in df_map.index:
      sub_lat = df_map.loc[i, "위도"]
      sub_long = df_map.loc[i, "경도"]
      title = df_map.loc[i, "번호"]
      typ = df_map.loc[i,"내진적용여부"]

      icon_color = "blue"
      if df_map.loc[i, "내진적용여부"] == "미적용":
          icon_color = "red"
      if df_map.loc[i, "내진적용여부"] == "부분적용":
          icon_color = "green"

      folium.Marker([sub_lat, sub_long], icon=folium.Icon(color=icon_color), popup=f'<i>{typ}</i>', tooltip=title).add_to(marker_cluster)

m.save('/content/gdrive/My Drive/DPL/강원도 전체 대피소.html')
m

## **느낀점**

 **** : 저번 수능 때 지진이 크게 나서 수능이 취소되었던 기억이 있기에 수능이 다가오는 지금 그때의 지진 이후로 우리나라에서 지진이 얼마나 일어났고 수능날이 다가오는 달에 지진이 크게 일어날 가능성이 있을지 그리고 만약 지진이 일어나면 어디로 대피해야할지에 대해 궁금해 우리나라에서의 지진 빈도수를 찾게되고 궁금해 이번 프로그래밍으로 조사하게 되었다.
 이번 활동을 통해 일본에서 가까운 지역인 경북, 경남에서 지진이 많이 발생했고 강원도는 생각보다 별로 지진이 안일어났다는 것을 알았고 춘천에서 지진이 일어나면 어디로 대피해야 할지도 알 수 있었다. 하지만 강원도에서 지정한 지진해일대피소에서 내진설계가 된 곳은 적었기 때문에 내진설계를 많이 적용하는 것이 시급하다고 생각한다.

 **** : 유용하게만 쓴다면 인터넷에 쌓아두는 데이터들도 중요하다는 것을 느꼈고 그런 데이터들을 프로그래밍으로  시각화하므로써 그냥 정보가 아닌 의미 있는 정보로 만드는것도 중요하다고 느꼈다. 과학에 대한 모든 이론들이 들어있는 책이 있다해도 우리가 이해하지 못하면(해석하지 못하면) 아무 소용이 없듯이 모든 데이터들도 우리가 읽기 힘들면 소용이 없기 때문이다.
컴퓨터가 데이터를-우리가 이해할 수 있는 데이터로-변환시키게 하기 위해서는 우리도 컴퓨터가 이해할 수 있게 코드를 잘 작성해야된다는 것을 만드는 과정에서 알게 됐다.